Read the message data into 'df' DataFrame, label columns (The given dataset here doesn't exist anymore so replace it with any other dataset)

In [3]:
import pandas as pd
import numpy as np

df=pd.read_table("https://raw.githubusercontent.com/justmarkham/pycon-2016-tutorial/master/data/sms.tsv",header=None)
names=['label','messages']
df.columns=names
df.head()

,label,messages
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


Convert values in 'label' column to numerical values and store in 'label_num' column for using MultinomialNB algorithm that only takes numerical data

In [5]:
df['label_num'] = df.label.map({'ham':0, 'spam':1})

The entire data is split 5 times into training and testing data such that each row of training data is once part of test data
The results of test data after 5 splits is combined according to test_index, i.e index of the rows of original data that came in test data. Then this combined predictions is compared with output values of original data

Count Vectorizer is called, Firstly the number of columns are decided based on number of disitnct words in the given data. 
Each row corresponds to a given message sentence and each column value will be the number of occurences of that distinct word (column name) in that sentence. This is all handled by CountVectorizer. 
 
CountVectorizer is internal to Jupyter Notebook, no need to import anything

The parameter to initialize CountVectorizer needs to be changed since it does not recognize single words like 'a' as distinct words. For this ' token_pattern=r"(?u)\b\w\w+\b" ' which is a default parameter is updated to ' token_pattern=r"(?u)\b\w+\b" ' which recognizes single words 

The vectorized data is converted to arrays for each message sentence

MultinomialNB library is used to find probabilities of spam given a distinct word (Laplace smoothing included in formula)
Attributes(distinct words) are indepenent -> Naive Bayes assumption
This library is useful for text data and natural language processing

In [6]:
from sklearn.feature_extraction.text import CountVectorizer

from sklearn.model_selection import KFold
kf = KFold(n_splits=5)

from sklearn.naive_bayes import MultinomialNB
nb = MultinomialNB()

results=pd.Series()

for train_index, test_index in kf.split(df):
    XTrain, XTest, YTrain, ytest = df['messages'].iloc[train_index], df['messages'].iloc[test_index], df['label_num'].iloc[train_index], df['label_num'].iloc[test_index]
    
    cv=CountVectorizer(stop_words=None, token_pattern=r"(?u)\b\w+\b")
    xtrain=cv.fit_transform(XTrain)
    xtest=cv.transform(XTest)
    nb.fit(xtrain,YTrain)
    predict_list = nb.predict(xtest)
    
    indexes=list(test_index)
    
    res=pd.Series(predict_list,index=indexes)
    results=results.append(res)
    
results=results.sort_index()           

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  if __name__ == '__main__':


In [7]:
from sklearn import metrics
print("Accuracy:",(metrics.accuracy_score(df['label_num'], results)*100),"%")

Accuracy: 98.56424982053123 %
